In [11]:
import torch
from torch import nn
from torch.nn import functional as F

n = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
n(X).shape

torch.Size([2, 10])

In [14]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


In [15]:
net = MLP()
net(X)

tensor([[-0.1346, -0.1589,  0.1344,  0.0257,  0.1351, -0.1732, -0.2400,  0.0982,
          0.2821, -0.0277],
        [-0.2426, -0.3856,  0.1191,  0.1110,  0.0142, -0.1053, -0.1040,  0.0655,
          0.3362,  0.0314]], grad_fn=<AddmmBackward0>)

In [16]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [17]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0980, -0.0961,  0.0090,  0.1422, -0.1203,  0.1569,  0.1576,  0.0303,
          0.1336,  0.1072],
        [ 0.0237, -0.1297,  0.0591,  0.1246, -0.0465,  0.0779,  0.1560,  0.0486,
          0.1235,  0.0049]], grad_fn=<AddmmBackward0>)

In [20]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.liner = nn.Linear(20, 20)

    def forward(self, X):
        X = self.liner(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1.)
        X = self.liner(X)
        while X.abs().sum() > 1.:
            X /= 2.
        return X.sum()

In [29]:
net = FixedHiddenMLP()
net(X)

tensor(0.4424, grad_fn=<SumBackward0>)

In [30]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.liner = nn.Linear(32, 16)

    def forward(self, X):
        return self.liner(self.net(X))

In [35]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.3546, grad_fn=<SumBackward0>)